Consulta o banco de dados e salva em csv cada tabela

In [ ]:
from difflib import *
from sqlalchemy import create_engine, text
import pandas as pd
from datetime import timedelta
import os
import win32com.client as win32

save_path = SAVE_PATH
# Database connection
engine = create_engine(
    DB_URL, 
    echo=False
)
#lista de nomes possíveis para as colunas
nomes_coluna = ["Impp", "IV Point[3f612]-i", "IV Point[a9112]-i"]

# Defina o limite de tempo
tempo_limite = pd.Timestamp.now().floor('s') - timedelta(minutes=60)
print(tempo_limite)

# Query to get table names
table_query = text(
    "SELECT tablename FROM pg_tables WHERE tablename LIKE '%_POS%' AND tablename NOT LIKE '%OLD%' AND schemaname = 'public'"
)

with engine.connect() as conn:
    # Execute the query to get table names
    table_names = conn.execute(table_query).fetchall()

    # Iterate through table names and export each to a CSV file
    for table_name in table_names:
        print(table_name)
        table_name = table_name[0]  # Get the table name string from the result

        # Create a flag to check if any column was found
        column_found = False

        # Iterate through possible column names
        for col_name in nomes_coluna:
            # Check if the column exists in the table
            query = text(f'SELECT column_name FROM information_schema.columns WHERE table_name = :table_name AND column_name = :col_name')
            result = conn.execute(query, {'table_name': table_name, 'col_name': col_name}).fetchone()
            
            if result:  # If the column exists
                query = text(f'SELECT "TIMESTAMP", "{col_name}" FROM "{table_name}"  WHERE "TIMESTAMP" >= :tempo_limite')
                df = pd.read_sql(query, conn, params={'tempo_limite': tempo_limite})  # Read the table into a DataFrame
                csv_file_name = f"\\{table_name.replace('/', '_')}.csv"
                final_save_path = save_path + csv_file_name        

                df.to_csv(final_save_path, index=False)  # Save DataFrame to CSV
                column_found = True  # Set flag to True

        # Check if no columns were found
        if not column_found:
            print(f"No matching columns found in table {table_name}")

print("CSV files have been created.")



Busca e apaga linhas duplicadas em cada arquivo

In [ ]:
# Caminho para a pasta contendo os arquivos csv
pasta = SAVE_PATH

# Lista todos os arquivos na pasta que terminam com .csv
arquivos = [arquivo for arquivo in os.listdir(pasta) if arquivo.endswith('.csv')]

# Processa cada arquivo
for arquivo in arquivos:
    caminho_completo = os.path.join(pasta, arquivo)
    df = pd.read_csv(caminho_completo)
    df = df.sort_values(by='TIMESTAMP', ascending=True).drop_duplicates(subset='TIMESTAMP', keep='first')
    # Salva o DataFrame de volta no arquivo CSV, se necessário
    df.to_csv(caminho_completo, index=False)
    # Ou armazene os DataFrames em uma lista se você precisar processá-los posteriormente
    # dataframes.append(df)


- Itera sobre cada arquivo e cria uma nova coluna para indicar se o valor é o mesmo da linha anterior
- Gera nova coluna Flag para indicar linhas de medidas travadas
- Separa arquivos travados em um novo df

In [ ]:
pasta = SAVE_PATH

resposta_travado = []
resposta_nao_travado = []

# Lista todos os arquivos CSV na pasta
arquivos = [arquivo for arquivo in os.listdir(pasta) if arquivo.endswith('.csv')]

def verificar_consecutivos_iguais(df, nome_arquivo):
    # Calcula a diferença entre valores consecutivos na coluna 'Corrente' e verifica se é igual a 0
    df["Flag"] = df['Corrente'].diff() == 0
    # Cria uma condição onde o valor atual na coluna 'Flag' é True e o valor anterior na coluna 'Flag' é True
    filterFreeze = df["Flag"]

    # Verifica se existem 2 ou mais linhas consecutivas com valor 'True' na coluna "Flag"
    if (filterFreeze.rolling(window=2).sum() >= 2).any():
        resposta_travado = f"Existem 2 ou mais pontos consecutivos com o mesmo valor no arquivo {nome_arquivo}"
        dado_travado = True
        resposta_nao_travado = None
    else:
        resposta_nao_travado = f"Não existem no mínimo 2 pontos consecutivos com o mesmo valor no arquivo {nome_arquivo}"
        dado_travado = False
        resposta_travado = None
    return resposta_travado, resposta_nao_travado, dado_travado

def processar_arquivos(arquivos, pasta, resposta_travado, resposta_nao_travado, verificar_consecutivos_iguais):
    # Itera sobre cada nome de arquivo na lista 'arquivos'
    for arquivo in arquivos:
        # Salva caminho do csv
        caminho_completo = os.path.join(pasta, arquivo)

        # Lê o arquivo CSV
        df = pd.read_csv(caminho_completo)

        # Separa o nome do arquivo e a extensão
        nome_arquivo, _ = os.path.splitext(arquivo)

        # Renomeia as colunas do DataFrame
        df.rename(columns={"Impp": "Corrente", "IV Point[3f612]-i": "Corrente", "IV Point[a9112]-i": "Corrente"}, inplace=True)

        # Verifica se existem 2 ou mais linhas consecutivas com valor 'True' na coluna "Flag"
        resposta_travado_msg, resposta_nao_travado_msg, dado_travado = verificar_consecutivos_iguais(df, nome_arquivo)

        if dado_travado:
            resposta_travado.append(resposta_travado_msg)
        else:
            resposta_nao_travado.append(resposta_nao_travado_msg)

        # Salva o DataFrame atualizado em CSV
        df.to_csv(caminho_completo, index=False)

# Chama a função para processar os arquivos
processar_arquivos(arquivos, pasta, resposta_travado, resposta_nao_travado, verificar_consecutivos_iguais)

# Imprime as respostas finais
# for resposta in resposta_travado:
#     print("Dado travado:", resposta)

# for resposta in resposta_nao_travado:
#     print("Dado não travado:", resposta)


Em cada arquivo, transforma TIMESTAMP, mantem apenas linhas onde df['flag] == True e adiciona nome do arquivo como a primeira coluna

In [ ]:
# Define o diretório contendo os arquivos CSV
pasta = SAVE_PATH

# Lista de arquivos a serem processados
arquivos = [f for f in os.listdir(pasta) if f.endswith('.csv')]

for arquivo in arquivos:
    # Salva o caminho do csv
    caminho_completo = os.path.join(pasta, arquivo)
    
    # Lê o arquivo CSV
    df = pd.read_csv(caminho_completo)

    # Separa o nome do arquivo e a extensão
    nome_arquivo, extensao = os.path.splitext(arquivo)

    # Garante que a coluna TIMESTAMP esteja no formato datetime
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])

    # Filtrar as linhas onde a coluna booleana é True
    df = df[df['Flag'] == True]

    #print(df['Flag'].value_counts())

    # Adiciona uma coluna com o nome do arquivo
    df.insert(0, "Arquivo", nome_arquivo)

    # Salva o DataFrame de volta no arquivo CSV, se necessário
    df.to_csv(caminho_completo, index=False)


Gera df_email gerando número de eventos (conjunto de minutos consecutivos) e contando o número de minutos travados para ser o df a ser enviado como alerta

In [ ]:
# Define o diretório contendo os arquivos CSV
pasta = SAVE_PATH

# Lista de arquivos a serem processados
arquivos = [f for f in os.listdir(pasta) if f.endswith('.csv')]

# DataFrame para armazenar os resultados
df_email = pd.DataFrame(columns=["Arquivo", "Eventos", "Tempo travado [minutos]"])

for arquivo in arquivos:
    # Salva o caminho do csv
    caminho_completo = os.path.join(pasta, arquivo)
    
    # Lê o arquivo CSV
    df = pd.read_csv(caminho_completo)

    # Separa o nome do arquivo e a extensão
    nome_arquivo, extensao = os.path.splitext(arquivo)
    
    # Garante que a coluna TIMESTAMP esteja no formato datetime
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    
    evento = 0
    ocorrencia = df.shape[0]
    
    for i in range(1, len(df)):
        diff = (df.loc[i, 'TIMESTAMP'] - df.loc[i-1, 'TIMESTAMP']).total_seconds()
        if diff != 60:
            evento += 1
            
    if evento > 0:
        evento += 1
        # Cria um DataFrame temporário para a nova linha
        new_row = pd.DataFrame([{
            "Arquivo": nome_arquivo,
            "Eventos": evento,
            "Tempo travado [minutos]": ocorrencia
        }])
        
        # Concatena o DataFrame temporário ao DataFrame df_email
        df_email = pd.concat([df_email, new_row], ignore_index=True)

# Salva o DataFrame de resultados em um novo arquivo CSV
#df_email.to_csv(os.path.join(pasta, "resultados.csv"), index=False)


Formata a tabela a ser enviada com os dados de df_email

In [ ]:
# Criar a integração com o Outlook
outlook = win32.Dispatch('outlook.application')

# Criar um email
email = outlook.CreateItem(0)

# Configurar as informações do seu e-mail
email.To = "acarolina066@gmail.com"
email.Subject = "Alerta Medidas"

df_email = df_email.to_html()

# Conteúdo HTML do corpo do e-mail
email.HTMLBody = f"""
<html>
<style>
  p, span {{font-size:14pt; text-align: left;
}}

  table, th, td {{font-size:14pt; border:1px solid black; border-collapse:collapse; text-align: center;
}}
  th, td {{padding: 10px;}}
</style>
<body>
      <p style="font-size:14pt; text-align: center;">Resumo Travamento dos minimódulos na última hora</p>
      <span>Legenda:
      </span>
      <p>Arquivo &rarr; Minimódulos que apresentaram pelo menos 1 minuto de travamento </p>
      <p>Eventos &rarr; Grupo de minutos consecutivos de travamento </p>
      <p>Tempo Travado &rarr; Tempo em minutos que o minimódulo ficou travado na útlima hora </p>
        {df_email}
        <strong><a href="http://150.162.142.219/viewdata" style="text-decoration: none;">PVBlocks</a> / 
        <a href="http://150.162.142.79:3000/d/nlTM5KzSk/shell-geral?orgId=1&viewPanel=13" style="text-decoration: none; color: orange;">Grafana</a>
        </strong>
</body>
</html>       
        """




# anexo = "C://Users/joaop/Downloads/arquivo.xlsx"
# email.Attachments.Add(anexo)

# Enviar o e-mail
email.Send()

print("Email Enviado")
